# CSUDH Chatbot Using RAGS and SQL DB

1. Chat Model
2. Chat Prompt Template
3. Output Parser
4. Initialize a embedding_model
5. Initialize a ChromaDB Connection
6. Retriever Object
7. RAG Chain
8. SQLChatMessageHistory
9. 

### **Step 1 - Import Chat Model and Configure the API Key**

In [1]:
# Step 1 - Import Chat Model and Configure the API Key

from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/openai_key.txt')
OPENAI_API_KEY = f.read()

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini", temperature=1)

## **Step 2 - Chat Prompt Template**

In [2]:
# Step 2 - Chat Prompt Template

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages(
    [
        # 1. System Prompt based on your old RAG use case
        SystemMessage(
            content = (""" You are chatbot for CSUDH to answer questions related to csudh ms in cs
            You are a factual assistant that answers strictly based on 
            1. first look for answers in the provided context and,
            2. second least priority is look in privided history of past messages, only if NER tagging is present, the prior history; do not use external knowledge, reasoning, or assumptions—if the answer exists in context or tagged history,
            return it exactly; if not, respond only with: "I’m sorry, I couldn’t find that information in the provided context."; 
            never infer, fabricate, or speculate, and ensure your response is 100% grounded in the given context and tagged history.
            """)
        ),
        
        # 2. Chat history placeholder
        MessagesPlaceholder(
            variable_name = "history"
        ),
        
        # 3. Human input formatted with RAG context
        HumanMessagePromptTemplate.from_template(
            "Context: {context}\n\nQuestion: {human_input}"
        ),
    ]
)


## **Step 3 - Initialize a Output Parser**

In [3]:
# Step 3 - Initialize a Output Parser

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

## **Step 4 - Initialize an embedding_model and ChromaDB Connection**

In [4]:
# Initialize an embedding_model

from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [5]:
# Initialize a ChromaDB Connection
from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created

db = Chroma(collection_name="vector_database", 
            embedding_function=embedding_model, 
            persist_directory="./chroma_db_")

## **Step - 6 Retriever Object**

In [6]:
# Converting CHROMA db connection to Retriever Object
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

## **Step 7 - SQLChatMessageHistory**

In [7]:
# SQLChatMessageHistory

# Create a connection with the database and 
# return the chat message history for a session id

from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    chat_message_history = SQLChatMessageHistory(
                                   session_id=session_id, 
                                   connection="sqlite:///chats_data/sqlite.db"
                               )
    return chat_message_history

## **Step 8 - Define a RAG Chain**

In [8]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter # Import itemgetter

In [9]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
# Define the core RAG chain without history first
rag_chain = (
    {
        "context": itemgetter("human_input") | retriever | format_docs,
        "human_input": RunnablePassthrough(),
        "history": itemgetter("history")
    }
    | chat_template
    | chat_model
    | parser
)

from langchain_core.runnables.history import RunnableWithMessageHistory
# Now, integrate the chat history
conversation_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_message_history_from_db,
    input_messages_key="human_input",
    history_messages_key="history",
)

In [11]:
# # This is where we configure the session id
# user_id = "Akhilmaurya2k@gmail.com"
# config = {"configurable": {"session_id": user_id}}

# input_prompt = {"human_input": "My name is Akhil. what are the office hours of dr. celly"}
# response = conversation_chain.invoke(input_prompt, config=config)

# print(response)

In [ ]:
# Step 8 - Run chat loop using RunnableWithMessageHistory

from langchain_core.messages import HumanMessage, AIMessage

# This is where we configure the session ID
user_id = "Akhilmaurya2k@gmail.com"
config = {"configurable": {"session_id": user_id}}

while True:
    # Step 1 - User input
    user_input = input("Enter your input (or type 'bye' to exit): ")
    print(f"*User: {user_input}")

    # Step 2 - Exit condition
    if user_input.lower() in ['bye', 'quit', 'exit']:
        print("*Session Ended.")
        break

    # Step 3 - Prepare input dict
    input_prompt = {"human_input": user_input}

    # Step 4 - Invoke the RAG conversation chain
    try:
        response = conversation_chain.invoke(input_prompt, config=config)
        print(f"*AI: {response}")
    except Exception as e:
        print(f"Error: {e}")


Enter your input (or type 'bye' to exit):  Hi


*User: Hi
*AI: I'm sorry, but I don’t have access to the specific system prompt given to me. If you have any questions or need information, feel free to ask!
